In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense

2024-04-28 17:59:47.470979: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:

#open the csv
data = pd.read_csv('data/csv/Ten_cities_with_info.csv')

def string_to_array(string):
    elements = string.strip('[]').split(',')
    array = np.array(elements, dtype=int)
    return array

data['Array'] = data['Array'].apply(string_to_array)

City_list = ['Zurich', 'Geneva', 'Basel', 'Lausanne', 'Bern', 'Winterthur', 'Luzern', 'St_Gallen', 'Lugano', 'Biel']
label_encoder = LabelEncoder()
data['Location'] = label_encoder.fit_transform(data['Location'])

print(data['Location'].value_counts())

Location
5    130
9    123
4    107
8    101
1     82
6     68
7     60
0     42
2     40
3     31
Name: count, dtype: int64


In [9]:

#split the dataset
X = np.array(data['Array'].to_list())
y = np.array(data['Location'].to_list())
East = np.array(data['East'].to_list())
North = np.array(data['North'].to_list())
sunshine_data = np.array(data['sunshine'].to_list())
rainfall_data = np.array(data['rainfall'].to_list())
temperature_data = np.array(data['temperature'].to_list())

X = X/255.0
sun_norm = (sunshine_data-np.min(sunshine_data))/(np.max(sunshine_data)-np.min(sunshine_data))
rain_norm = (rainfall_data-np.min(rainfall_data))/(np.max(rainfall_data)-np.min(rainfall_data))
temp_norm = (temperature_data-np.min(temperature_data))/(np.max(temperature_data)-np.min(temperature_data))
East = (East-np.min(East))/(np.max(East)-np.min(East))
North = (North-np.min(North))/(np.max(North)-np.min(North))

X_complete = np.column_stack((East, North))
X_complete = X_complete[:, np.newaxis]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_complete, y, test_size=0.3)

shape = X_complete[0].shape

model = Sequential()
model.add(Flatten(input_shape=shape))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Activation('softmax'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(10, activation= 'softmax'))

#compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

/Users/edwinvincendeau/Desktop/2eme semestre/Advanced/ADA/Swisscities/.venv/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
#train the model
model.fit(X_train, y_train, epochs=10)

#test accuracy
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/10


/Users/edwinvincendeau/Desktop/2eme semestre/Advanced/ADA/Swisscities/.venv/lib/python3.10/site-packages/keras/src/backend/tensorflow/nn.py:625: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1152 - loss: 2.3819
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1833 - loss: 2.2138
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1656 - loss: 2.2241
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1526 - loss: 2.2092
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1525 - loss: 2.2083
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1611 - loss: 2.2017 
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1604 - loss: 2.2240 
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1755 - loss: 2.2086
Epoch 9/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1578 - loss: 2.2236 
Epoch 10/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1555 - loss: 2.2249
8/8 - 0s - 23ms/step - accuracy: 0.1483 - loss: 2.2064

Test accuracy: 0.14830508828163147


In [12]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

cm = confusion_matrix(y_test, y_pred)
print(cm)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[ 0  0  0  0  0  0  0  0  0  9]
 [ 0  0  0  0  0  0  0  0  0 22]
 [ 0  0  0  0  0  0  0  0  0 15]
 [ 0  0  0  0  0  0  0  0  0 10]
 [ 0  0  0  0  0  0  0  0  0 39]
 [ 0  0  0  0  0  0  0  0  0 43]
 [ 0  0  0  0  0  0  0  0  0 17]
 [ 0  0  0  0  0  0  0  0  0 18]
 [ 0  0  0  0  0  0  0  0  0 28]
 [ 0  0  0  0  0  0  0  0  0 35]]


In [3]:
df = pd.read_csv('data/csv/Images_with_canton.csv')
df = df[df['Kanton'!= 'foreign']]
print(df['Kanton'].value_counts())

KeyError: True

In [ ]:
def string_to_array(string):
    elements = string.strip('[]').split(',')
    array = np.array(elements, dtype=int).reshape((100,100))
    return array

data['Array'] = data['Array'].apply(string_to_array)

label_encoder = LabelEncoder()
data['Kanton'] = label_encoder.fit_transform(data['Kanton'])